# Testing the Limits of Pre-training

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# %env PYTHONPATH="../"
sys.path.append('../')

In [19]:
PATH = '../../scratch/aorl/e2e_maze_8_2/e2e_maze_8_2.a5406616b3951230fafa4ca948138750f5881270649f6138153e795abde02629'
RUN_NAME = 'e2e_maze_8_2.a5406616b3951230fafa4ca948138750f5881270649f6138153e795abde02629'
PROJECT_NAME = 'aorl'

## Get W&B Information

In [25]:
import wandb

api = wandb.Api()
# wandb.login(key='wandb_v1_Ly3eFqIQrxItHDUO00wHBVWktDM')

In [30]:
ENTITY = 'jnzhao3'

run = api.runs(f'{ENTITY}/{PROJECT_NAME}', filters={'display_name': f'{RUN_NAME}'})[0]

In [33]:
run.config['seed']

3

## Load the Originally Trained Agent

In [10]:
from utils.flax_utils import restore_agent, save_agent
from agents import agents

In [13]:
from agents.gcfql import get_config

In [14]:
config = get_config()

In [15]:
agent_class = agents[config['agent_name']]

In [36]:
from utils.datasets import GCDataset, Dataset
import pathlib
from wrappers.datafuncs_utils import make_env_and_datasets

DATA_STEP = 3000000

dataset_path = pathlib.Path(PATH) / f'data-{DATA_STEP}.npz'
env, train_dataset, val_dataset = make_env_and_datasets(run.config['env_name'], dataset_path=str(dataset_path), use_oracle_reps=True)
dataset_class = GCDataset
train_dataset = dataset_class(Dataset.create(**train_dataset, freeze=False), config)

In [37]:
save_dir = PATH
step = 1000000

example_batch = train_dataset.sample(1)
agent = agent_class.create(
    run.config['seed'],
    example_batch,
    config
)

agent = restore_agent(agent, save_dir, step)

Restored from ../../scratch/aorl/e2e_maze_8_2/e2e_maze_8_2.a5406616b3951230fafa4ca948138750f5881270649f6138153e795abde02629/params_1000000.pkl
